In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import os

In [22]:
title = '글짓기'

In [27]:
file_path = 'C:/Users/User/Desktop/024.에세이 글 평가 데이터/01.데이터/1.Training/라벨링데이터/TL_글짓기/글짓기'  # 백슬래시를 슬래시로 변경

json_list = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith(".json")]

# JSON 파일 불러오기 및 확인
data_list = []
for json_file in json_list:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
        data_list.append(data)

len(data_list)  # JSON 파일의 개수 확인

4574

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import json

# 전처리된 데이터 파일 경로
file_path = 'C:/Users/User/Desktop/024.에세이 글 평가 데이터/01.데이터/1.Training/라벨링데이터/TL_글짓기/글짓기/preprocessed_essays.json'

# 데이터 불러오기
with open(file_path, 'r', encoding='utf-8') as file:
    essays_data = json.load(file)

# 데이터 전처리: 입력 데이터와 타겟 데이터 준비
input_data = []  # 전처리된 입력 데이터
target_data = []  # 채점 점수(라벨) 데이터

# 실제 데이터셋에 따라 적절한 형태로 전처리 수행
for essay_id, essay_content in essays_data.items():
    # 여기에 데이터를 입력 형식에 맞게 가공하는 코드를 작성합니다.
    # 예시로 데이터를 불러오고 더미 채점 점수(0~5)를 사용합니다.
    input_data.append(essay_content)  # 전처리된 에세이 데이터를 입력 데이터로 사용
    # 채점 점수는 실제 데이터에서 적절한 값으로 대체되어야 합니다.
    target_data.append(np.random.randint(0, 6))  # 더미 채점 점수(0~5) 생성

# 데이터를 numpy 배열로 변환
input_data = np.array(input_data)
target_data = np.array(target_data)

# 훈련 데이터와 검증 데이터로 분할
train_input, val_input, train_target, val_target = train_test_split(
    input_data, target_data, test_size=0.2, random_state=42
)

# 데이터를 PyTorch의 Tensor로 변환
train_input_tensor = torch.from_numpy(train_input).float()
train_target_tensor = torch.from_numpy(train_target).long()
val_input_tensor = torch.from_numpy(val_input).float()
val_target_tensor = torch.from_numpy(val_target).long()

# 데이터셋 및 데이터 로더 생성
train_dataset = TensorDataset(train_input_tensor, train_target_tensor)
val_dataset = TensorDataset(val_input_tensor, val_target_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # LSTM의 마지막 타임스텝 출력만 사용
        return out

# 모델 생성 및 손실 함수, 최적화 기법 설정
input_size = len(input_data[0])  # 입력 크기 설정 (에세이 데이터의 특성 수)
output_size = 6  # 출력 크기 설정 (채점 점수 범위)
hidden_size = 64  # LSTM의 hidden unit 크기 설정
model = LSTMModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 훈련
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # 검증 데이터셋을 사용한 모델 평가
    model.eval()
    val_running_loss = 0.0
    for inputs, targets in val_loader:
        outputs = model(inputs)
        val_loss = criterion(outputs, targets)
        val_running_loss += val_loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader)} - Val Loss: {val_running_loss/len(val_loader)}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/Desktop/024.에세이 글 평가 데이터/01.데이터/1.Training/라벨링데이터/TL_글짓기/글짓기/preprocessed_essays.json'

In [28]:
type_list = []
count_correction = 0
for i in range(4574):
    with open(json_list[i]) as data_file:
        sample_data = json.load(data_file)
        try:
            # dictionary 형태로 되어있는 경우
            if sample_data['correction']['sentence_corr_txt_grammar'] == '':
                pass
            else:
                print('t:',i)
        except:
            # 배열 형태로 되어있는 경우
            #print(sample_data['correction'])
            count_correction+=1
            for j in range(len(sample_data['correction'])):
                now_sample = sample_data['correction'][j]
                
                for k in range(len(now_sample['sentence_corr_type'])):
                    corr_type = now_sample['sentence_corr_type'][k]
                    corr_detail_type = now_sample['sentence_corr_detail_type'][k]
                    corr_reason = now_sample['sentence_corr_reason'][k]
                    type_list.append(f'{corr_type}_{corr_detail_type}_{corr_reason}')
                    if f'{corr_type}_{corr_detail_type}_{corr_reason}' == '문법_어미_':
                        print(now_sample)
                        print()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 52: illegal multibyte sequence

In [29]:
json_list[0]

'C:/Users/User/Desktop/024.에세이 글 평가 데이터/01.데이터/1.Training/라벨링데이터/TL_글짓기/글짓기\\글짓기_고등_1학년_ESSAY_33982.json'